## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [1]:
from TCGame_Env1 import TicTacToe #import your class from environment file
import collections
import numpy as np
from numpy import nan
import random
import pickle
import time
from matplotlib import pyplot as plt

In [2]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [3]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(state):
    
    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [4]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0.0

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [5]:
# Defining epsilon-greedy policy. You can choose any function epsilon-decay strategy
def epsilon_greedy(state, time):
    epsilon = - 1/ (1 + np.exp((-time+7000)/1700)) + 1
    z = np.random.random()
        
    if z > epsilon:
        state = Q_state(state)
        action = max(Q_dict[state],key=Q_dict[state].get) 
        return action#Exploitation: this gets the action corresponding to max q-value of current state
    else:
        action_avail = valid_actions(state)
        if len(action_avail) > 0:
            random = np.random.choice(len(action_avail))
            action = action_avail[random]   #Exploration: randomly choosing and action
            return action
        
    

#### Tracking the state-action pairs for checking convergence - write your code here

In [6]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)
Q_dict = collections.defaultdict(dict)
States_track = collections.defaultdict(dict)

print(len(Q_dict))
print(len(States_track))

0
0


In [15]:
# Initialise states to be tracked
def initialise_tracking_states():
  
    sample_q_values = [(('x-x-x-x-x-x-x-x-x'),(7,5)),
                      (('3-6-4-1-7-8-x-x-x'),(8,5)),
                      (('5-x-x-x-x-x-x-4-x'),(2,7)),
                      (('8-5-4-7-x-2-3-9-6'),(4,1))]    #select any 4 Q-values
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []    #this is an array which will have appended values of that state-action pair        

In [8]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [9]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action])

In [16]:
initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [11]:
#Defining parameters for the experiment

# EPISODES = 20
EPISODES = 50000        
LR = 0.01            #learning rate
GAMMA = 0.85


threshold = 2000       #every these many episodes, the 4 Q-values will be stored/appended (convergence graphs)
# threshold = 1
policy_threshold = 3000    #every these many episodes, the Q-dict will be updated
# policy_threshold = 3

### Q-update loop ---write your code here

In [12]:
# start_time = time.time()
win,tie,lost,total_reward = [0,0,0,0]

for episode in range(0,EPISODES):
    ##### Start writing your code from the next line
    env = TicTacToe()      #creating an instance of the class
    
    #get initial state
    curr_state = env.state     
    
    add_to_dict(curr_state)
    
    actions_avail = valid_actions(curr_state) #get available actions
    random = np.random.choice(len(actions_avail))
    curr_action = actions_avail[random] #choosing a random action
        
    terminal_val = False
    
    while terminal_val is not True:    #run as long as the game is not over. Eg: terminal state not achieved
        
        curr_action = epsilon_greedy(curr_state, episode)
                
        Q_curr_state = Q_state(curr_state)
                        
        #take action based on action returned from the epsilon greedy strategy
        next_state, reward, terminal = env.step(curr_state, curr_action)

        add_to_dict(next_state)
        
        #Q-value update sequence 
        if terminal[0] == False:

            # UPDATE RULE
            Q_next_state = Q_state(next_state)
            max_next = max(Q_dict[Q_next_state],key=Q_dict[Q_next_state].get)   #this gets the action corresponding to max q-value of next state
            Q_dict[Q_curr_state][curr_action] += LR * ((reward + (GAMMA*(Q_dict[Q_next_state][max_next]))) - Q_dict[Q_curr_state][curr_action] ) 
            curr_state = next_state       #state(t) became state(t-1)
            total_reward += reward
        
        terminal_val = terminal[0]
        
    #Tracking the agent wins, losses and ties
    if reward == 10:
        win += 1
    elif reward == 0:
        tie += 1
    else:
        lost += 1
    
    #TRACKING Q-VALUES
    if ((episode+1) % threshold) == 0:   
        save_tracking_states()
        save_obj(States_track,'States_tracked') 
    
    #SAVING POLICY
    if ((episode+1)% policy_threshold ) == 0:  
        save_obj(Q_dict,'Policy')  
    #Check Progress
    if ((episode+1) % 1000) == 0:
        print('Games Played',episode)
            
# elapsed_time = time.time() - start_time
print('Reward',total_reward)
print('Stats:Win-',win,'Tie',tie,"loss",lost)
save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')

('Games Played', 999)
('Games Played', 1999)
('Games Played', 2999)
('Games Played', 3999)
('Games Played', 4999)
('Games Played', 5999)
('Games Played', 6999)
('Games Played', 7999)
('Games Played', 8999)
('Games Played', 9999)
('Games Played', 10999)
('Games Played', 11999)
('Games Played', 12999)
('Games Played', 13999)
('Games Played', 14999)
('Games Played', 15999)
('Games Played', 16999)
('Games Played', 17999)
('Games Played', 18999)
('Games Played', 19999)
('Games Played', 20999)
('Games Played', 21999)
('Games Played', 22999)
('Games Played', 23999)
('Games Played', 24999)
('Games Played', 25999)
('Games Played', 26999)
('Games Played', 27999)
('Games Played', 28999)
('Games Played', 29999)
('Games Played', 30999)
('Games Played', 31999)
('Games Played', 32999)
('Games Played', 33999)
('Games Played', 34999)
('Games Played', 35999)
('Games Played', 36999)
('Games Played', 37999)
('Games Played', 38999)
('Games Played', 39999)


KeyboardInterrupt: 

#### Check the Q-dictionary

In [ ]:
Q_dict

In [ ]:
len(Q_dict)

In [13]:
#reading the tracked Q-values
infile = open('States_tracked.pkl','rb')
new_dict = pickle.load(infile)
new_dict

defaultdict(dict,
            {'3-6-4-1-7-8-x-x-x': {(8, 5): []},
             '5-x-x-x-x-x-x-4-x': {(2, 7): [0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               -0.01,
               -0.01,
               -0.01,
               -0.01]},
             '8-5-4-7-x-2-3-9-6': {(4, 1): []},
             'x-x-x-x-x-x-x-x-x': {(7, 5): [-0.4188335858818902,
               -0.5993153470484593,
               -0.7372874127497715,
               -0.8311778434193093,
               -0.8925977942573625,
               -0.9309820965002912,
               -0.9560918115149285,
               -0.9717840886529704,
               -0.982049446724955,
               -0.9884647662016163,
               -0.9926614516791151,
               -0.9952841637065404,
       

In [18]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated
best_action = max(Q_dict['x-x-x-x-x-x-x-x-x'],key=Q_dict['x-x-x-x-x-x-x-x-x'].get)
print(best_action)

(7, 9)


#### Check the states tracked for Q-values convergence
(non-evaluative)

### Epsilon - decay check

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,5000000)
epsilon = []
for i in range(0,5000000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()